<a href="https://colab.research.google.com/github/wjdqlsdlsp/AI_using_pytorch/blob/main/%EA%B3%BC%EC%A0%9C4_Cliff_Walking_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[인공지능] 과제4 Cliff Walking 예제 구현**
*   **QLearning Class 및 Sarsa Class를 완성하여 결과를 살펴보는 것이 목표**입니다.
*   기본적인 코드는 아래 노트에 모두 작성되어 있습니다. 비어있는 함수 부분을 완성하면 됩니다.
*   **과제 수행 시 주의사항: 외부 라이브러리로 Q-learning 및 Sarsa 적용하지 말 것, 수업 때 배운 내용대로 Q-learning과 Sarsa를 주어진 함수에 구현할 것.** 웹 상에 있는 다양한 Q-learning 및 Sarsa 코드를 참고하는 것은 괜찮습니다.
*   **보고서 작성 내용**: 여러분이 완성한 Q-learning 및 sarsa 알고리즘의 내용과 결과의 의미를 분석하는 내용을 작성하면 됩니다.
작성한 코드와 실행 결과를 첨부하길 바라며, 코드에는 자세한 주석을 필수적으로 포함하기 바랍니다. 보고서는 PDF로 제출바랍니다.
*   보고서는 11월 30일 오후 11시 59분까지 블랙보드에 보고서 형태로 제출하면 됩니다. 지각은 0점입니다.

# **본 노트를 본인의 drive로 복사하여 활용하기 바랍니다.**

본 과제는 OpenAI Gym 환경에 기반하여 작성되었습니다. Gym 라이브러리는 학습을 적용할 수 있는 다양한 환경을 제공합니다. 여기서는 수업에서 다뤘던 Cliff Walking 환경을 활용합니다.

In [220]:
import gym
import matplotlib
import numpy as np
import random
import itertools
import sys
from collections import defaultdict
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv # Cliff Walking 환경

QtoPolicy Class는 학습된 Q-value를 입력하면 해당하는 Q-value의 greedy 정책이 출력되도록 하는 함수 `printPolicy`를 구성하는 Class입니다. Q-learning 및 Sarsa를 이용하여 학습된 정책을 출력하기 위해 필요합니다.

In [221]:
class QtoPolicy:
    def __init__(self):
        self.action = ['↑','→','↓','←']
    
    def printPolicy(self, Q):
        policy = np.array([np.argmax(Q[key]) if key in Q else -1 for key in np.arange(48)])
        v = ([np.max(Q[key]) if key in Q else 0 for key in np.arange(48)])
        actions = np.stack([self.action for _ in range(len(policy))], axis=0)
        
        print(np.take(actions, np.reshape(policy, (4, 12))))
        print('')

아래는 Q-learning 알고리즘을 수행하는 Class의 정의입니다.
*   `update()` 메쏘드의 경우 state, action, reward, next_state, next_action이 주어졌을 때 Q-value를 업데이트하는 함수입니다.
*   `act()` 메쏘드의 경우 $\epsilon$-greedy 정책에 따라 action을 선택하는 함수입니다.



In [222]:
class QLearning:
    # 초기값 설정
    def __init__(self):
        self.action_no = 4 # 가능한 액션 갯수
        self.alpha = 0.01 # 반영률
        self.gamma = 0.9 # Discount factor
        self.epsilon = 0.2 # 입실론 (액션을 취하는 확률)
        self.q_values = defaultdict(lambda: [0.0] * self.action_no) # Q테이블

    # 업데이트 함수 ( 학습 )
    def update(self, state, action, reward, next_state, next_action): 
        q_value = self.q_values[state][action] # 현재 state에서의 q_value값
        next_q_value = max(self.q_values[next_state]) # 다음상태에서 q_value값 (탐욕적 행동을 결정하기에 max 값을 가짐)
        td_error = (reward + self.gamma * next_q_value) - q_value # target 정책과, 행동의 정책과의 value 차이
        self.q_values[state][action] = q_value + self.alpha * td_error # q_vlaue 업데이트 (반영률을 통해 값 반영)
    
    # 행동 함수
    def act(self, state):
        '''
        설정한 입실론 값보다, 작을 경우 조건문 참조건이 실행되며, 랜덤으로 가능한 동작 중 하나를 선택합니다.
        그 외의 경우, 현재 state가 가지는 모든 Q값 중 가장 큰 값을 가지는 행동을 선택합니다.
        '''
        return np.random.choice(self.action_no) if np.random.rand() < self.epsilon else np.argmax(self.q_values[state]) # 행동 반환

아래는 Sarsa 알고리즘을 수행하는 Class의 정의입니다.
*   `update()` 메쏘드의 경우 state, action, reward, next_state, next_action이 주어졌을 때 Q-value를 업데이트하는 함수입니다.
*   `act()` 메쏘드의 경우 $\epsilon$-greedy 정책에 따라 action을 선택하는 함수입니다.



In [223]:
class Sarsa:
    def __init__(self):
        # 초기값 설정
        self.action_no = 4 # 가능한 액션 갯수
        self.alpha = 0.01 # 반영률
        self.gamma = 0.9 # Discount factor
        self.epsilon = 0.2 # 입실론 (액션을 취하는 확률)
        self.q_values = defaultdict(lambda: [0.0] * self.action_no)
        
    # 업데이트 함수 ( 학습 )
    def update(self, state, action, reward, next_state, next_action):
        q_value = self.q_values[state][action] # 현재 state에서 q_value값
        next_q_value = self.q_values[next_state][next_action] # 다음상태에서 q_value값 (Sarsa의 경우 behavior 정책 사용)
        td_error = (reward + self.gamma * next_q_value) - q_value # target 정책과, 행동의 정책과의 value 차이
        self.q_values[state][action] = q_value + self.alpha * td_error # q_value 업데이트 (반영률 통해 값 반영)

    # 행동 함수    
    def act(self, state):
        '''
        설정한 입실론 값보다, 작을 경우 조건문 참조건이 실행되며, 랜덤으로 가능한 동작 중 하나를 선택합니다.
        그 외의 경우, 현재 state가 가지는 모든 Q값 중 가장 큰 값을 가지는 행동을 선택합니다.
        '''
        return np.random.choice(self.action_no) if np.random.random() < self.epsilon else np.argmax(self.q_values[state])

OpenAI Gym에서의 Cliff Walking 환경을 로드하고 해당하는 환경을 살펴보기 위해 `render()` 메쏘드를 사용해봅니다.
그리고 `env.nS` 및 `env.nA` 변수를 통해 해당 환경의 state 및 action 개수를 확인합니다.

Cliff Walking 환경에서 각 state는 grid에서의 위치, action은 'up', 'right', 'down', 'left' 방향을 의미합니다.

In [224]:
env = CliffWalkingEnv()
env.render()
print ('Number of states: ', env.nS)
print ('Number of actions :', env.nA)

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Number of states:  48
Number of actions : 4


주어진 Q-value에서 greedy policy를 출력하는 QtoPolicy Class를 정의합니다.

In [225]:
policy = QtoPolicy()

Q-learning Class를 정의하고 5000 episode 동안 학습을 수행합니다.

Gym 라이브러리의 환경에서는 `step(action)` 메쏘드를 통해 해당하는 time-step에서 action을 수행한 효과를 얻을 수 있습니다. 해당 메쏘드에서는 action을 수행하여 얻어지는 보상 (reward), 다음 상태 (next_state), done (episode 종료여부) 등이 출력으로 주어집니다.

In [226]:
agent_QL = QLearning()
for ep in range(5000):
    done = False
    state = env.reset()
    action = agent_QL.act(state)
    
    ep_rewards = 0
    while not done:
        next_state, reward, done, info = env.step(action)

        next_action = agent_QL.act(next_state)

        agent_QL.update(state, action, reward, next_state, next_action)
        
        ep_rewards += reward
        state = next_state
        action = next_action
    if (ep+1) % 200 == 0:
        print(f"episode: {ep+1}, rewards: {ep_rewards}")

episode: 200, rewards: -256
episode: 400, rewards: -296
episode: 600, rewards: -139
episode: 800, rewards: -144
episode: 1000, rewards: -144
episode: 1200, rewards: -360
episode: 1400, rewards: -25
episode: 1600, rewards: -33
episode: 1800, rewards: -19
episode: 2000, rewards: -15
episode: 2200, rewards: -17
episode: 2400, rewards: -29
episode: 2600, rewards: -128
episode: 2800, rewards: -35
episode: 3000, rewards: -15
episode: 3200, rewards: -15
episode: 3400, rewards: -33
episode: 3600, rewards: -15
episode: 3800, rewards: -14
episode: 4000, rewards: -123
episode: 4200, rewards: -131
episode: 4400, rewards: -17
episode: 4600, rewards: -323
episode: 4800, rewards: -18
episode: 5000, rewards: -26


Sarsa에 대해서도 같은 방식으로 학습을 수행합니다.

In [227]:
agent_Sa = Sarsa()
for ep in range(5000):
    done = False
    state = env.reset()
    action = agent_Sa.act(state)
    
    ep_rewards = 0
    while not done:
        next_state, reward, done, info = env.step(action)

        next_action = agent_Sa.act(next_state)

        agent_Sa.update(state, action, reward, next_state, next_action)
        
        ep_rewards += reward
        state = next_state
        action = next_action
    if (ep+1) % 200 == 0:
        print(f"episode: {ep+1}, rewards: {ep_rewards}")

episode: 200, rewards: -484
episode: 400, rewards: -85
episode: 600, rewards: -45
episode: 800, rewards: -138
episode: 1000, rewards: -22
episode: 1200, rewards: -27
episode: 1400, rewards: -23
episode: 1600, rewards: -26
episode: 1800, rewards: -150
episode: 2000, rewards: -27
episode: 2200, rewards: -20
episode: 2400, rewards: -19
episode: 2600, rewards: -17
episode: 2800, rewards: -122
episode: 3000, rewards: -19
episode: 3200, rewards: -22
episode: 3400, rewards: -140
episode: 3600, rewards: -15
episode: 3800, rewards: -17
episode: 4000, rewards: -22
episode: 4200, rewards: -22
episode: 4400, rewards: -19
episode: 4600, rewards: -27
episode: 4800, rewards: -119
episode: 5000, rewards: -23


학습된 Q-value를 이용하여 학습된 정책을 출력합니다.

In [228]:
print('Learned policy by Q-learning')
policy.printPolicy(agent_QL.q_values)
print('Learned policy by Sarsa')
policy.printPolicy(agent_Sa.q_values)

Learned policy by Q-learning
[['←' '↓' '→' '↓' '→' '→' '↑' '↓' '→' '↓' '→' '↓']
 ['↑' '→' '↑' '↑' '→' '↓' '→' '→' '→' '→' '→' '↓']
 ['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' '↑']]

Learned policy by Sarsa
[['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '↑' '↑' '↑' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '↑' '↑' '↑' '↑' '↑' '↑' '↑' '→' '→' '→' '↓']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' '↑']]



In [229]:
env.close()